<a href="https://colab.research.google.com/github/Arbiej/Inheritance/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install eli5


     |████████████████████████████████| 112kB 3.0MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance
from ast import literal_eval
from tqdm import tqdm_notebook

In [0]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
ls data


men_shoes.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
def run_model(feats, model= DecisionTreeRegressor(max_depth=5)):
  X = df [ feats ].values
  y = df ['prices_amountmin'].values
  scores = cross_val_score(model,X,y,scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
df['brand_cat']= df['brand'].map(lambda x: str(x).lower()).factorize ()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

run_model(['brand_cat'])

model =RandomForestRegressor(max_depth =5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

In [0]:
df.head()

In [0]:
model =RandomForestRegressor(max_depth =5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [0]:
df.features.head().values


array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [188]:
test={'key': 'value'}
test['key']

str(test)

"{'key': 'value'}"

In [0]:
str_dict= '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'
literal_eval(str_dict)[0]['value'][0]

'Men'

In [0]:
def parse_features(x):
  output_dict={}
  if str(x) == 'nan' : return output_dict

  features = literal_eval(x.replace('\\"','"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()
    
    output_dict[key] = value

  return output_dict

df['features_parsed' ] = df['features'].map(parse_features)


In [0]:
keys = set()

df['features_parsed'].map(lambda x: keys.update(x.keys()))

len (keys)


476

In [130]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats [key] if key in feats else np.nan)

In [0]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_rise', 'feat_catalog', 'feat_athlete', 'feat_product type',
       'feat_best sellers rank', 'feat_age start', 'feat_age gender group',
       'feat_occasion', 'feat_brand', 'feat_case type'],
      dtype='object', length=527)

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100



---



{}

In [137]:
{k:v for k,v in keys_stat.items() if v > 35}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296}

In [0]:
df['feat_brand_cat']= df['feat_brand'].factorize ()[0]
df['feat_color_cat']= df['feat_color'].factorize ()[0]
df['feat_gender_cat']= df['feat_gender'].factorize ()[0]
df['feat_manufacturer part number_cat']= df['feat_manufacturer part number'].factorize ()[0]
df['feat_material_cat']= df['feat_material'].factorize ()[0]

df['feat_sport_cat']= df['feat_sport'].factorize ()[0]
df['feat_style_cat']= df['feat_style'].factorize ()[0]

for key in keys:
  df[get_name_feat(key) + '_cat']= df[get_name_feat(key)].factorize ()[0]
  


In [139]:
df['brand']=df['brand'].map(lambda x: str(x).lower())
df[df.brand == df.feat_brand ].shape

(8846, 532)

In [0]:
feats= [' ']

In [0]:
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

In [183]:
df['weight'].unique()

array([nan, '3.0 lbs', '9 g', '1.45 lbs', '0.45 lbs', '1.0 lbs',
       '0.23 lbs', '5.0 lbs', '5.5 lbs', '7.45 lbs', '4.0 lbs',
       '2.7969 lbs', '3.9 lbs', '4.6 pounds', '2.1 lbs', '1.1057 lbs',
       '15.0 lbs', '2.4 ounces', '454 g', '0.105 lbs', '9.1 ounces',
       '4.8 lbs', '6.1 lbs', '6.5 lbs', '1.1041 lbs', '1.3 Kg', '91 g',
       '20.0 lbs', '6.0 lbs', '386 g', '0.81 lbs', '4.5 lbs',
       '0.5 ounces', '2.0 lbs', '3.13 lbs', '5.9 lbs', '6.15 lbs',
       '1 pounds', '1.95 lbs', '2.15 lbs', '2 pounds', '2.1 pounds',
       '14 Kg', '0.4788 lbs', '10.0 lbs', '0.38 lbs', '2.5 lbs',
       '68.912 lbs', '45 g', '13.09 lbs', '2.5 pounds', '0.21 lbs',
       '16.75 lbs', '6.3 lbs', '272 g', '1.8 Kg', '2.8 pounds', '0.1 lbs',
       '5.05 lbs', '0.28 lbs', '76.08 lbs', '0.15 lbs', '200 g',
       '7.8 pounds', '399 g', '4.95 lbs', '64.144 lbs', '24 pounds',
       '73.696 lbs', '1.6 lbs', '6.6 ounces', '5 g', '1.2 Kg', '862 g',
       '3.05 lb', '8.6 ounces', '3.6 lbs', '71.

In [0]:

feats = ['brand_cat','feat_shape_cat','feat_metal type_cat', 'feat_brand_cat','feat_gender_cat','feat_material_cat', 'feat_sport_cat', 'feat_style_cat' ]

model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)

In [198]:
X = df[ feats ].values
y = df['prices_amountmin'].values
m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

(-57.2051114576365, 4.160348940426072)


Weight,Feature
0.2573 ± 0.0077,brand_cat
0.1033 ± 0.0075,feat_material_cat
0.0246 ± 0.0025,feat_gender_cat
0.0177 ± 0.0017,feat_brand_cat
0.0127 ± 0.0011,feat_shape_cat
0.0089 ± 0.0012,feat_metal type_cat
0.0030 ± 0.0004,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [167]:
df[ df['brand'] == 'nike'].features_parsed.sample(5).values

array([{},
       {'gender': 'men', 'size': 'adult 10.5', 'clothing size': 'adult 10.5', 'brand': 'nike', 'color': 'gray/volt'},
       {'sport': 'running shoes', 'gender': 'men', 'shoe size': '9.5 men us', 'shoe category': 'running shoes;training shoes;lifestyle;casual shoes', 'color': 'cstl blue/bl sprk/white/pht bl', 'model': '705149 403', 'casual & dress shoe style': 'running shoes;training shoes;lifestyle;casual shoes', 'manufacturer part number': '705149 403', 'variant group id': '224#mp#705149 403', 'brand': 'nike', 'age group': 'men'},
       {},
       {'sport': 'training shoes', 'occasion': 'training shoes', 'material': 'synthetic-and-mesh', 'gender': 'men', 'shoe size': '9.5 us men', 'shoe category': "men's shoes", 'assembled product dimensions (l x w x h)': '12.00 x 8.00 x 6.00 inches', 'fabric content': 'synthetic-and-mesh', 'color': 'poison gree/hyper blue/white', 'model': '553684 302', 'shoe closure': 'lace-up', 'casual & dress shoe style': 'cross trainer shoes', 'manufa

In [170]:
 df['feat_age group'].value_counts()

adult               4563
men                  350
child                 77
men's                 33
unisex                 6
infant                 4
toddler                4
mens                   4
boys'                  3
men||women             2
women                  2
youth                  2
women ,�� unisex       2
12 up                  1
adult ,�� teen         1
Name: feat_age group, dtype: int64

In [202]:
!git commit -m "day 5"


*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@7e6a9c8be01a.(none)')


In [0]:
!git config --global user.email"bartlomiej.rudnicki@gmail.com"
!git config --global user.name "Bartek"

In [204]:
!git commit -m"day5"


*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@7e6a9c8be01a.(none)')


In [0]:
!git config --global user.email "bartlomiej.rudnicki@gmail.com"
!git config --global user.name "Bartek"

In [208]:
!git commit -m "day5"

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
	modified:   HelloGithub.ipynb
	deleted:    matrix_one/day3.ipynb

Untracked files:
	ColabDriveGithub.ipynb
	matrix_one/day4_meta.ipynb

no changes added to commit


In [209]:
!git push


Everything up-to-date


In [210]:
!git add origin master

fatal: pathspec 'origin' did not match any files


In [211]:
!git add

Nothing specified, nothing added.
Maybe you wanted to say 'git add .'?


In [212]:
!git add.

git: 'add.' is not a git command. See 'git --help'.

The most similar command is
	add
